https://hipster4020.tistory.com/176

In [12]:

### Service 
def blue(str):
    from Functions import Service
    return Service.colored_text(str,'blue')
def yellow(str):
    from Functions import Service
    return Service.colored_text(str,'yellow')
def red(str):
    from Functions import Service
    return Service.colored_text(str,'red')

## 자연어처리 패키지 설치 
def NLPInstalls():
    import subprocess,sys
    # pip가 없으면 pip를 설치
    try:import pip
    except ImportError:
        print("Install pip for python3")
        subprocess.call(['sudo', 'apt-get', 'install', 'python3-pip'])
    
    # tweepy 없으면 tweepy 설치
    try:import tweepy        
    except ModuleNotFoundError:
        print("Install tweepy")
        subprocess.call([sys.executable, "-m", "pip", "install", 'tweepy==3.10.0'])
    finally:import tweepy 
    
    # konlpy 없으면 konlpy 설치
    try:import konlpy
    except ModuleNotFoundError: 
        print("Install konlpy")
        subprocess.call([sys.executable, "-m", "pip", "install", 'konlpy'])
    finally:import konlpy
    
    # eunjeon 없으면 eunjeon 설치
    try:import eunjeon
    except ModuleNotFoundError: 
        print("Install eunjeon : eunjeon")
        subprocess.call([sys.executable, "-m", "pip", "install", 'eunjeon'])
    finally:import konlpy
    
    # datasets 없으면 datasets를 설치
    try:import datasets
    except ModuleNotFoundError: 
        print("Install eunjeon : datasets")
        subprocess.call([sys.executable, "-m", "pip", "install", 'datasets'])
    finally:import datasets
    
    # pytorch 없으면 pytorch 설치
    try:import torch
    except ModuleNotFoundError: 
        print("Install eunjeon : pytorch")
        subprocess.call([sys.executable, "-m", "pip", "install", 'pytorch'])
    finally:import torch
    
    # transformers 없으면 transformers 설치
    try:import transformers
    except ModuleNotFoundError: 
        print("Install eunjeon : transformers")
        subprocess.call([sys.executable, "-m", "pip", "install", 'transformers'])
    finally:import transformers
NLPInstalls()


## 감정분석 데이터 셋 준비 
from datasets import load_dataset
emotions = load_dataset("emotion")

In [15]:
## pytorch importing 
import torch
import torch.nn.functional as F
import warnings ; warnings.filterwarnings('ignore')
from transformers import AutoTokenizer
# from transformers import DistilBertTokenizer

## chkpt :  Hugging face 에 등록된 distilBert 언어 모델을 가져옴 .
model_ckpt = 'distilbert-base-uncased' 
print(yellow("🔸 model check point : "), model_ckpt)
## 토크나이저를 통해서 토큰화 작업.
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

## pretrained model  을 사용하려면 automodel importing,
from transformers import AutoModel
## pytorch 를 사용하여 GPU 사용 여부를 확인 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
## 사용하는 device 에 맞는 모델을 자동으로 가져옴. 
print(yellow("🔸 사용하는 device : "),device)
model = AutoModel.from_pretrained(model_ckpt).to(device)
print(yellow("🔸 사용하는 디바이스에 맞는 모델: "),model)

🔸 model check point :  distilbert-base-uncased
🔸 사용하는 device :  cpu
🔸 사용하는 디바이스에 맞는 모델:  DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
   

In [9]:
## frame work 호호환 예시
def framework():
    from transformers import TFAutoModel
    tf_model = TFAutoModel.from_pretrained(model_ckpt)
    ## 프레임워크간 상호 호환이 되지않음, pytorch 와 tensorflow 의 체크포인트가 다 다르다.!!
    tf_xlmr = TFAutoModel.from_pretrained('xlm-roberta-base', from_pt =True)
    ## 이런식으로 from_pt 를 붙이면 라이브러리가 자동으로 파이토치 가중치를 다운로드해 변환함. 



Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


> 마지막 은닉 상태 추출하기 
- 하나의 문자열에 대한 마지막 은닉 상태를 추출 함. 
- 문자열을 인코딩하고 토큰을 파이토치 텐서로 변환함. 
- tokenizer 에 return_tensor="pt" 매개변수를 지정해 이작업을 수행함. 

In [42]:

text = 'this is a test i am Forrest'
inputs = tokenizer(text,return_tensors ='pt') ## 파이토치텐서로 변환
print(blue(f'원본 text : '),red(f'{text}'))
print(yellow(f'🔸  입력 텐서 크기 :{inputs["input_ids"].size()} = [bach_size, n_tokens]'))
print(yellow(f'🔸  Ids(vector) : {inputs["input_ids"]}'))
print(yellow(f'🔸  Attention mask : {inputs["attention_mask"]}'))

inputs

원본 text :  this is a test i am Forrest
🔸  입력 텐서 크기 :torch.Size([1, 9]) = [bach_size, n_tokens]
🔸  Ids(vector) : tensor([[  101,  2023,  2003,  1037,  3231,  1045,  2572, 16319,   102]])
🔸  Attention mask : tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])


{'input_ids': tensor([[  101,  2023,  2003,  1037,  3231,  1045,  2572, 16319,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}

> device 에 있는 모델에 입력을 전달



In [45]:
inputs

{'input_ids': tensor([[  101,  2023,  2003,  1037,  3231,  1045,  2572, 16319,   102]]),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [44]:
## k -> input_ids,attenition_mask v -> tensor, tensor
inputs = {k:v.to(device) for k,v in inputs.items()} 

with torch.no_grad(): ## 컨텍스트 매니저 사용.  메모리양 줄어 추론할때 유리함. 
    outputs = model(**inputs)
    # 모델은 은닉상태 하나만 반환함. 
print(outputs)

BaseModelOutput(last_hidden_state=tensor([[[ 0.0185,  0.0394,  0.0283,  ..., -0.0326,  0.2196,  0.3650],
         [-0.1387, -0.3454, -0.0797,  ..., -0.1926,  0.3445,  0.2568],
         [-0.1854, -0.1602,  0.1703,  ...,  0.1295,  0.2070,  0.7750],
         ...,
         [ 0.1707,  0.2024,  0.1363,  ...,  0.1139,  0.2042,  0.1838],
         [ 0.3101, -0.1290, -0.0113,  ...,  0.4925, -0.1913,  0.0816],
         [ 0.9054,  0.2051, -0.3114,  ...,  0.1355, -0.6349, -0.3079]]]), hidden_states=None, attentions=None)


In [46]:
outputs.last_hidden_state.size()
# 은닉상태 텐ㅅ서 크기 
# [batch_size, n_tokens, hidden_dim ] -> 768 개의 벡터가 생성됨 

torch.Size([1, 9, 768])

In [53]:
outputs.last_hidden_state[:,0].size()
# outputs.last_hidden_state[:,0] ## 문자열 첫 문자의 은닉상태 를 출력


torch.Size([1, 768])

In [ ]:
def 